### import the data for training

In [1]:
import numpy
import tensorflow as tf
import h5py
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from sklearn.cross_validation import StratifiedKFold
from keras.layers import Dropout
from keras.regularizers import l2
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import Convolution2D
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
import pandas as pd
from sklearn.cross_validation import train_test_split
import scipy.misc
from scipy import misc
import numpy as np

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
x=[]
for i in range(1,10):
    arr = scipy.misc.imread('imgs/imgs/'+str(i)+'.jpg',flatten='True',mode='L')
    
    #arr = scipy.misc.imresize(arr,size=(96,96))
    #scipy.misc.imshow(arr)
    #arr = arr.reshape((1,200,200))
    x.append(arr)
x1 = np.asarray(x)
y = [0,0,0,0,1,1,1,0,1]
y1 = np.asarray(y)
y1 = y1.reshape(9,1)

In [2]:
x = pd.read_csv('fer2013/fer2013.csv')
newx = x.loc[x['emotion'].isin((3,4,6))]
X = newx['pixels']
Y = newx['emotion']
t = []
for i in X:
    t.append(map(int, i.split(' ')))
XFull = np.asarray(t)
Y = np.asarray(Y)
Y = Y.reshape(Y.shape[0],1)
print XFull.shape
print Y.shape
Y[Y==3] = 0
Y[Y==4] = 1
Y[Y==6] = 2

(21264, 2304)
(21264, 1)


In [3]:
import os
happy = []
folder = './happy_faces_cropped'
suffix ='jpg'
for fn in os.listdir(folder):
    if(fn.endswith(suffix)):
        arr = scipy.misc.imread(folder+'/'+str(fn),flatten='True')
        arr = arr.flatten()
        happy.append(arr)
happyFinal = np.asarray(happy)
happyLabel = np.zeros(len(happy))
happyLabel = happyLabel.reshape(happyLabel.shape[0],1)
print happyFinal.shape
print happyLabel.shape

(811, 2304)
(811, 1)


In [5]:
import os
neutral = []
folder = './neutral_faces_cropped'
suffix ='jpg'
for fn in os.listdir(folder):
    if(fn.endswith(suffix)):
        arr = scipy.misc.imread(folder+'/'+str(fn),flatten='True')
        arr = arr.flatten()
        neutral.append(arr)
neutralFinal = np.asarray(neutral)
neutralLabel = np.zeros(len(neutral))
for i in range(len(neutralLabel)):
    neutralLabel[i] = 2
neutralLabel = neutralLabel.reshape(neutralLabel.shape[0],1)
print neutralFinal.shape
print neutralLabel.shape
print neutralLabel[0]

(671, 2304)
(671, 1)
[ 2.]


In [4]:
import os
sad = []
folder = './sad_faces_cropped'
suffix ='jpg'
for fn in os.listdir(folder):
    if(fn.endswith(suffix)):
        arr = scipy.misc.imread(folder+'/'+str(fn),flatten='True')
        arr = arr.flatten()
        sad.append(arr)
sadFinal = np.asarray(sad)
sadLabel = np.ones(len(sad))
sadLabel = sadLabel.reshape(sadLabel.shape[0],1)
print sadFinal.shape
print sadLabel.shape
print sadLabel[0]

(200, 2304)
(200, 1)
[ 1.]


In [6]:
XFull = np.concatenate((XFull,happyFinal,neutralFinal,sadFinal),axis = 0)
Y = np.concatenate((Y,happyLabel,neutralLabel,sadLabel),axis = 0)
print XFull.shape
print Y.shape

(22946, 2304)
(22946, 1)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(XFull,Y,test_size=0.3, random_state=42)
X_train = X_train.reshape(X_train.shape[0], 1,48,48).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1,48,48).astype('float32')

### Normalise the data

In [8]:
X_train = X_train / 255
X_test = X_test / 255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

### Create Model

In [9]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Convolution2D(128, 7, 7, border_mode='valid', input_shape=(1, 48, 48), activation='relu'))#54
    #	model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(64, 5, 5, activation='relu'))#50
    model.add(MaxPooling2D(pool_size=(2, 2)))#25
    model.add(Convolution2D(64, 5, 5, activation='relu'))#21
    model.add(Convolution2D(64, 3, 3, activation='relu'))#19
    model.add(MaxPooling2D(pool_size=(2, 2)))#9
    model.add(Dropout(0.2))
    model.add(Flatten())
    #	model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

### Train and Predict

In [10]:
model = larger_model()
#model.load_weights("weights4_layers_updated_11layer_9700_wts.h5",by_name=False)
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=15, batch_size=300, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))
model.save("weights4_layers_Jokebot_50_alldataepochs_new.h5")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (7, 7), padding="valid", activation="relu", input_shape=(1, 48, 48...)`
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), activation="relu")`
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), activation="relu")`
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 16062 samples, validate on 6884 samples
Epoch 1/15
298s - loss: 1.0794 - acc: 0.4280 - val_loss: 1.0957 - val_acc: 0.4208
Epoch 2/15
298s - loss: 1.0681 - acc: 0.4329 - val_loss: 1.0591 - val_acc: 0.4362
Epoch 3/15
299s - loss: 1.0062 - acc: 0.4978 - val_loss: 0.9297 - val_acc: 0.5601
Epoch 4/15
300s - loss: 0.9060 - acc: 0.5671 - val_loss: 0.8773 - val_acc: 0.5960
Epoch 5/15
300s - loss: 0.8587 - acc: 0.5971 - val_loss: 0.8634 - val_acc: 0.5978
Epoch 6/15
299s - loss: 0.8316 - acc: 0.6170 - val_loss: 0.8413 - val_acc: 0.6119
Epoch 7/15
300s - loss: 0.7978 - acc: 0.6366 - val_loss: 0.8249 - val_acc: 0.6206
Epoch 8/15
299s - loss: 0.7703 - acc: 0.6529 - val_loss: 0.8139 - val_acc: 0.6331
Epoch 9/15
299s - loss: 0.7398 - acc: 0.6700 - val_loss: 0.8098 - val_acc: 0.6291
Epoch 10/15
300s - loss: 0.7275 - acc: 0.6810 - val_loss: 0.7938 - val_acc: 0.6429
Epoch 11/15
300s - loss: 0.6852 - acc: 0.7033 - val_loss: 0.7868 - val_acc: 0.6447
Epoch 12/15
300s - loss: 0.6587 - acc: 0.7146 -

### Take test data and predict

In [ ]:
xPred = numpy.fromfile('dataset/test_x.bin', dtype='uint8')
xPred = xPred.reshape((20000,96,96))
xPred[xPred<255] = 0
xPred = xPred/ 255
#xPred[xPred<1] = -1
xPred=xPred.reshape(xPred.shape[0],1,96,96).astype('float32')

In [ ]:
x1 = x1.reshape(x1.shape[0], 1,96,96).astype('float32')

In [ ]:
from scipy import misc
arr = misc.imread('Images/000001.jpg') # 640x480x3 array
print arr.shape
print arr[20, 30] # 3-vector for a pixel
print arr[20, 30, 1] # green value for a pixel
print type(arr)
#scipy.misc.imshow(arr)
#arr = arr.flatten()
print arr.shape
#arr = arr.reshape((600,600,3))

#arr[arr<255] = 0
arr = scipy.misc.imread('Images/000001.jpg',flatten='True',mode='L')
scipy.misc.imshow(arr)
print arr.shape
print arr[0:100]

arr = scipy.misc.imresize(arr,size=(200,200))
#arr = arr.flatten()
#scipy.misc.imshow(arr)
print arr.shape
print arr[0:100]
arr = arr.reshape((1,200,200))
#print arr[0:100]
#print arr.shape
#scipy.misc.imshow(arr)

In [ ]:
import numpy as np
#import tensorflow as tf
#tf.python.control_flow_ops = tf
import scipy.misc # to visualize only
x = numpy.fromfile('Images/000001.bin', dtype='uint8')
print x.shape

x = x.reshape((3,600,600))
#x[x<255] = 0

scipy.misc.imshow(x[0]) # to visualize only

### import all libraries

In [ ]:
import numpy
#import h5py
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from sklearn.cross_validation import StratifiedKFold
from keras.layers import Dropout
from keras.regularizers import l2
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import Convolution2D
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
import pandas as pd
from sklearn.cross_validation import train_test_split

In [ ]:
x = arr
y = [1]

In [ ]:
num_pixels = x.shape[0] * x.shape[1]

In [ ]:
#y=y.as_matrix()
x1 = x1.reshape(x1.shape[0], 1,200,200).astype('float32')
#X_test = X_test.reshape(X_test.shape[0], 1,60,60).astype('float32')

In [ ]:
print x1[0]

### the input data has been transformed to the form 1,60,60 

In [ ]:
y = pd.read_csv('dataset/train_y.csv')
y=y['Prediction']
y=y.as_matrix()
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state=42)
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], 1,60,60).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1,60,60).astype('float32')

### normalise the input

In [ ]:
X_train = X_train / 255
X_test = X_test / 255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

### create the model with the different convolutions 

## 9 Layer Architecture

In [ ]:
def larger_model():
	# create model
	model = Sequential()
	model.add(Convolution2D(32, 7, 7, border_mode='valid', input_shape=(1, 60, 60), activation='relu'))
	model.add(Dropout(0.2))
#	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Convolution2D(32, 5, 5, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Convolution2D(64, 5, 5, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Convolution2D(64, 3, 3, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Convolution2D(128, 5, 5, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Convolution2D(128, 3, 3, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Flatten())
	model.add(Dropout(0.2))
#	model.add(Dense(256, activation='relu'))
	model.add(Dense(256, activation='relu'))
	model.add(Dense(128, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

## 6-Layer Architecture 

In [ ]:
def larger_model():
	# create model
	model = Sequential()
	model.add(Convolution2D(64, 7, 7, border_mode='valid', input_shape=(1, 60, 60), activation='relu'))#54
#	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Convolution2D(64, 5, 5, activation='relu'))#50
	model.add(MaxPooling2D(pool_size=(2, 2)))#25
	model.add(Convolution2D(64, 5, 5, activation='relu'))#21
	model.add(Convolution2D(64, 3, 3, activation='relu'))#19
	model.add(MaxPooling2D(pool_size=(2, 2)))#9
	model.add(Dropout(0.2))
	model.add(Flatten())
#	model.add(Dense(256, activation='relu'))
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

## 11-Layer Architecture

In [ ]:
def larger_model():
	# create model
	model = Sequential()
	model.add(Convolution2D(64, 3, 3, border_mode='valid', input_shape=(1, 60, 60), activation='relu'))#58
	model.add(Convolution2D(64, 3, 3, activation='relu'))#56
	model.add(MaxPooling2D(pool_size=(2, 2)))#28
	model.add(Convolution2D(64, 3, 3, activation='relu'))#26
	model.add(Convolution2D(64, 3, 3, activation='relu'))#24
	model.add(MaxPooling2D(pool_size=(2, 2)))#12
	model.add(Convolution2D(64, 3, 3, activation='relu'))#10
	model.add(Convolution2D(64, 3, 3, activation='relu'))#8
	model.add(MaxPooling2D(pool_size=(2, 2)))#4
	model.add(Convolution2D(64, 3, 3, activation='relu'))#2
	model.add(MaxPooling2D(pool_size=(2, 2)))#1
	model.add(Dropout(0.3))
	model.add(Flatten())
	model.add(Dense(128, activation='relu',W_regularizer=l2(0.01)))
	model.add(Dense(50, activation='relu',W_regularizer=l2(0.01)))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

### Train the model and evaluate

In [ ]:
model = larger_model()
#model.load_weights("weights4_layers_updated_11layer_9700_wts.h5",by_name=False)
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=5, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))
#model.save("weights4_layers_updated_11layer_part3.h5")

### Take test data and predict output 

In [ ]:
xPred = numpy.fromfile('dataset/test_x.bin', dtype='uint8')
xPred = xPred.reshape((20000,60,60))
xPred[xPred<255] = 0
xPred = xPred/ 255
#xPred[xPred<1] = -1
xPred=xPred.reshape(xPred.shape[0],1,60,60).astype('float32')

In [ ]:
scipy.misc.imshow(xPred[19962]) # to visualize only

### Can save the model and reuse it for future purposes 

In [ ]:
model.save("weights4_layers_updated_2.h5")

In [27]:
from keras.models import load_model
model = load_model("weights4_layers_updated_11layer_part3.h5")

In [ ]:
model.save_weights("weights4_layers_updated_11layer_9704_wts.h5")

In [ ]:
model.load_weights("weights4_layers_wts.h5",by_name=False)

## Predict for the test data

In [10]:
#xPred.shape
predictions = model.predict(X_test,verbose=0)

In [11]:
finalpred=[]
for i in predictions:
    finalpred.append([j for j,z in enumerate(i) if z == max(i)])

In [22]:
newy_test = []
for i in y_test:
    newy_test.append([j for j,z in enumerate(i) if z == max(i)])

In [23]:
count =0
for i in range(len(finalpred)):
    if(finalpred[i] == newy_test[i]):
        count+=1

print (count * 1.0)/len(finalpred)

0.648024404416


In [26]:
print newy_test[0:30]
print finalpred[0:30]

[[1], [1], [1], [0], [0], [2], [0], [2], [0], [1], [1], [1], [2], [0], [2], [0], [0], [0], [2], [1], [0], [1], [1], [0], [0], [1], [1], [0], [2], [0]]
[[0], [0], [0], [0], [0], [2], [0], [0], [0], [0], [1], [1], [0], [0], [1], [1], [0], [0], [0], [1], [1], [1], [0], [0], [0], [1], [0], [0], [2], [0]]


### Final output into a file as per the required format 

In [ ]:
import pandas as pd
predictionFileName = "Predictions4layers_updated_11layer_9700.csv"
result = pd.DataFrame(columns=('Id','Prediction'))
count = 0
for i in finalpred:
    y = "%d"%count
    value = "%d"%i[0]
    result.loc[count]=[y,value]
    count+=1
result.to_csv(predictionFileName,index=False)

In [ ]:
import csv

with open("finalpred_modified.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(finalpred[0:15000])

with open("finalpred0.8_modified1.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(finalpred[15000:20000])